markstro
3/27/2020

This notebook is for estimating the parameter "dday_slope" for GF v1.1



Compute the daily radadj value for each HRU for each day

In [ ]:
import numpy as np
import pandas as pd
import fiona 
import geopandas as gpd
import math
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [ ]:
nrel_shapefile_fn = "c:/Users/markstro/work1.1/GIS/nhm_shapefiles/nhm_hru_gf1_1_nrel_solrad.shp"

# This file produced by jupyter notebook "solar_table". It produces the same values as the soltab.f90 PRMS module.
soltab_solt_fn = 'c:/Users/markstro/work1.1/soltab/soltab_solt_GF_v1.1.csv'

# radadj is the output of this notebook
radadj_fn = 'c:/Users/markstro/work1.1/soltab/dday_slope_radadj_GF_v1.1.csv'

# This is the hru_slope value from the PRMS parameter file
hru_slope_fn = 'c:/Users/markstro/work/input/hru_slope.csv'

gdb_path = 'c:/Users/markstro/work1.1/GIS/GFv1.1_v2e.gdb'

# This is the hru_slope value from the PRMS parameter file
hru_elev_fn = 'c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/hru_elev.csv'
hru_slope_fn = 'c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/hru_slope.csv'
hru_aspect_fn = 'c:/Users/markstro/work1.1/paramdb_v1.1/paramdb_master/hru_aspect.csv'

In [ ]:
coolwarm = cm.get_cmap('coolwarm', 12)
print(coolwarm)

# Next blocks load in the monthly solrad targets from NREL Direct Normal Irradiance from http://nrel.gov/gis/solar.html

In [ ]:
# Read in the monthly short wave data from NREL. It's in the shapefile that was created by running zonal means
# on the monthly DNI geotifs downloaded from NREL.

hrus = gpd.read_file(nrel_shapefile_fn)

In [ ]:
hrus.head()

In [ ]:
# Get the order of the nhru_v11 IDs for mapping the soltab values.

nhru_v11_vals = hrus["nhru_v11"]
print(nhru_v11_vals)

In [ ]:
# For whatever reason, the zonal mean process from QGIS left some NaNs when filling in the Direct Normal Irradiance (dni)
# monthly values in the shapefile (see dni* columns above). Those need to be filled in with real values.
# Without going back to the GIS (which I already ran with the results that are shown above), I am using np.interpolate across
# each column. This is a hack in the sense that the adjacent row do not necessarily mean that the HRUs are adjacent, and it
# is uncertain exactly what is being interpolated, but it is filling in the nan values with real values and allows
# me to move on.


print(hrus.shape)

print("original coordinates with nan value")
count = 0
for ii in range(hrus.shape[0]):
    for jj in range(8,20):
        if np.isnan(hrus.iloc[ii,jj]):
            print(" (", ii,jj, ")", end = '')
            count += 1
print("")
print(count)
      

for ii in range(hrus.shape[0]):
    for jj in range(8,20):
        hrus.iloc[:,jj] = hrus.iloc[:,jj].interpolate()

print("subsequent coordinates with nan value")
count = 0
for ii in range(hrus.shape[0]):
    for jj in range(8,20):
        if np.isnan(hrus.iloc[ii,jj]):
            print(" (", ii,jj, ")", end = '')
            count += 1

print("")
print(count)


In [ ]:
# Join the HRU slopes from the PRMS parmaeter file (paramdb) to the features from the shapefile.

hru_slope_df = pd.read_csv(hru_slope_fn)
hrus = hrus.set_index('nhru_v11').join(hru_slope_df.set_index('$id'))

In [ ]:
# Take the cosine of the hru slopes. The hru_slope must be converted from rise/run to radians with arctan first.

hrus['hru_cossl'] = np.cos(np.arctan(hrus['hru_slope']))
hru_cossl_vals = hrus['hru_cossl'].values

In [ ]:
hrus.head()

In [ ]:
# Extract the NREL monthly solrad targets from the dataframe into a numpy 2D array for the radadj calculation below.

solrad_targets_vals= hrus[["dni_jan_me", "dni_feb_me", "dni_mar_me", "dni_apr_me", "dni_may_me", "dni_jun_me",
                    "dni_jul_me", "dni_aug_me", "dni_sep_me", "dni_oct_me", "dni_nov_me", "dni_dec_me"]].values
print(solrad_targets_vals)

In [ ]:
# The NREL solrad values are in units of kWh/m2/Day. PRMS uses Langleys per day. The conversion factor comes from
# https://www.wcc.nrcs.usda.gov/ftpref/wntsc/H&H/GEM/SolarRadConversion.pdf

#                    W-sec    1 KW     1 hour               KW-hours
# 1 Langley = 41868 -------  ------   -------  =   0.6978  ---------
#                     m2     1000 W   60 sec                  m2

solrad_targets_vals_langleys = solrad_targets_vals / 0.6978
print(solrad_targets_vals_langleys)

In [ ]:
# read the soltab values from the csv file produced by solar_table.ipynb

soltab_df = pd.read_csv(soltab_solt_fn, header=None)
soltab_df.head()

In [ ]:
# soltab values from the csv file produced by solar_table.ipynb

soltab_df_vals = soltab_df.values
print(soltab_df_vals)
print(soltab_df_vals.shape)

In [ ]:
# Find all the values less than 10.0 and set them to 10.0. That gives something for the
# radajd calculation something to work with.

count = np.sum(soltab_df_vals < 10.0)
print(count)
print(float(count)/float(366.0 * 114958.0))

soltab_df_vals[soltab_df_vals < 10.0] = 10.0

print(soltab_df_vals)

In [ ]:
# create an array (len = 366, number of days in the year) that for any jday, it gives the month index.
jan = [0] * 31
feb = [1] * 29 # assume leap year to get full table
mar = [2] * 31
apr = [3] * 30
may = [4] * 31
jun = [5] * 30
jul = [6] * 31
aug = [7] * 31
sep = [8] * 30
octo = [9] * 31
nov = [10] * 30
dec = [11] * 31

month_of_jday = jan + feb + mar + apr + may + jun + jul + aug + sep + octo + nov + dec
print(len(month_of_jday), month_of_jday)

In [ ]:
# Compute radadj for all days-of-the-year and for all HRUs

radadj = np.zeros(soltab_df.shape)
nday = soltab_df.shape[0]
nhru = soltab_df.shape[1]

min_count = 0
max_count = 0
for jday in range(nday):
    imon = month_of_jday[jday]
    for ihru in range(nhru):
        kk = nhru_v11_vals[ihru] - 1
        try:
            radadj[jday,ihru] = solrad_targets_vals_langleys[ihru,imon] * hru_cossl_vals[ihru] / soltab_df_vals[jday,kk]
        except:
            print(jday, ihru, imon, solrad_targets_vals_langleys[ihru,imon], hru_cossl_vals[ihru], soltab_df_vals[jday,kk])
            
        if radadj[jday,ihru] < 0.05:
            radadj[jday,ihru] = 0.05
            min_count += 1
            
        if radadj[jday,ihru] > 0.95:
            radadj[jday,ihru] = 0.05
            max_count += 1
        
print ((jday * nhru), min_count, max_count)

In [ ]:
jday = 366

hrus["solrad_targets_vals_langleys"] = solrad_targets_vals_langleys[:,month_of_jday[jday-1]]
print(len(hrus["solrad_targets_vals_langleys"]))

print(solrad_targets_vals_langleys.shape)
print(min(solrad_targets_vals_langleys[:,month_of_jday[jday-1]]),max(solrad_targets_vals_langleys[:,month_of_jday[jday-1]]))

print(min(hrus["solrad_targets_vals_langleys"]),max(hrus["solrad_targets_vals_langleys"]))

plt.hist(solrad_targets_vals_langleys[:,month_of_jday[jday-1]])
plt.show()

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# df.dropna(thresh=2)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
hrus.plot(column='solrad_targets_vals_langleys', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# df.dropna(thresh=2)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
hrus.plot(column='hru_cossl', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
hrus["radadj"] = radadj[jday - 1,:]

In [ ]:
# plot the radadj values

# plot all HRUs in white (background color)
f, ax = plt.subplots(1, figsize=(12, 12))
#hrus.plot(color="white", ax=ax)

# df.dropna(thresh=2)

# plot the HRUs with valid temperature (non-nan) with their color ramp colors
hrus.plot(column='radadj', cmap='coolwarm', ax=ax, legend=True)

In [ ]:
print(hrus)

In [ ]:
# Write the radadj values out to csv file so all of the stuff above doesn't need to be rerun to complete the next steps.

radadj_t = np.transpose(radadj)
cols = list(range(1, 367))
#print(cols)
df = pd.DataFrame(data=radadj_t, columns=cols)
df["nhru_v11"] = nhru_v11_vals
df.head()

In [ ]:
# Write the radadj values out to csv file so all of the stuff above doesn't need to be rerun to complete the next steps.

df.to_csv(path_or_buf=radadj_fn)